In [ ]:

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
class PerlinNoiseGenerator:
    def __init__(self, n_iter, timescale, noise_factor: int = 1):
        self.baseline  = 20
        self.scaling   = 5
        self.n_iter    = n_iter
        self.timescale = timescale

    def fade(self, t):
        """Fade function as defined by Ken Perlin."""
        return t * t * t * (t * (t * 6 - 15) + 10)

    def lerp(self, a, b, x):
        """Linear interpolation."""
        return a + x * (b - a)

    def grad(self, hash_value, x):
        """Gradient function."""
        h = hash_value & 15
        grad = 1 + (h & 7)  # Gradient value is one of 1, 2, ..., 8
        return (grad * x)  # Compute the dot product
 
    def generate(self):
        # Create a permutation array
        p = np.arange(256, dtype=int)
        np.random.shuffle(p)
        p = np.stack([p, p]).flatten()  # Duplicate to avoid overflow

        def perlin(x):
            """Generate Perlin noise for input x."""
            X = int(x) & 255
            x -= int(x)
            u = self.fade(x)
            
            a = p[X]
            b = p[X + 1]

            return self.lerp(self.grad(a, x), self.grad(b, x - 1), u)
        
        # Example of using the Perlin noise generator to produce temperature variations
        time_steps = np.linspace(0, self.timescale, self.n_iter)
        outdoor_temperatures = [20 + self.scaling*perlin(t) for t in time_steps]
        return time_steps, outdoor_temperatures

time_steps, outdoor_temps = PerlinNoiseGenerator(10001, 10, 1).generate()

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 3))
plt.plot(time_steps, outdoor_temps)
plt.show()

In [ ]:
# Constants
class Environment: 
    TEMPERATURE_GOAL = 21.0  # desired temperature in Celsius
    ACTION_SCALE = 5.0  # amount of heating/cooling applied per action unit
    
    def __init__(self, outdoor_temps, init_temp):
        self.outdoor_temps = outdoor_temps
        self.curr_outdoor_temp = outdoor_temps[0]
        self.curr_indoor_temp  = init_temp
        self.curr_time         = 0
    
    def run_timestep(self, power):
        """Update indoor temperature based on outdoor temperature and applied power."""
        # Here, 0.05 is an arbitrary coefficient showing influence of outdoor temp on indoor temp
        # 0.1 is another coefficient showing influence of power
        # (heating/cooling) on indoor temp
        self.curr_indoor_temp = self.curr_indoor_temp \
            + 0.05 * (self.curr_outdoor_temp - self.curr_indoor_temp) + \
            + 0.10 * power \
            + np.random.normal(0, 0.2)
        
        self.curr_time += 1
        self.curr_outdoor_temp = self.outdoor_temps[self.curr_time]
            
        return self.curr_time, self.curr_indoor_temp, self.curr_outdoor_temp

In [ ]:
from dataclasses import dataclass
@dataclass
class TimeStepData:
    time: int
    orig_indoor_temp: float
    orig_outdoor_temp: float
    pred_indoor_temp: float = None
    next_indoor_temp: float = None
    action: float           = None
    power: float            = None 
    free_energy: float      = None
    
    def detach(self):
        for attr_name in self.__dataclass_fields__:
            attr = getattr(self, attr_name)
            if isinstance(attr, torch.Tensor):
                setattr(self, attr_name, attr.detach())
        return self


In [ ]:
# Define RNN model
class ThermostatRNN(nn.Module):
    def __init__(self, optimal_temp = 21.0, lr=0.001):
        super(ThermostatRNN, self).__init__()
        self.rnn = nn.RNN(input_size=3, hidden_size=20, num_layers=1, batch_first=True)  # input size increased to 3
        self.fc = nn.Linear(20, 2)
        self.optimal_temp = optimal_temp
        
        # Learning Functions
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.criterion = nn.MSELoss()


    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.fc(x)
        return x

    def free_energy(self, predicted, observed):
        """Calculate the free energy, which is the discrepancy between predicted and observed."""
        return self.criterion(predicted, observed)

    def train_step(self, data: TimeStepData):
        """Training step for the RNN."""
        self.optimizer.zero_grad()
        inpt = [data.orig_indoor_temp, data.orig_outdoor_temp, data.action]
        predicted = model(torch.FloatTensor([inpt]).unsqueeze(0))
        predicted_next_temp = predicted[0, 0, 0]
        loss = self.free_energy(predicted_next_temp, data.next_indoor_temp)
        loss.backward()
        self.optimizer.step()
        return loss

    def choose_action(self, data: TimeStepData):
        orig_indoor_temp, orig_outdoor_temp = data.orig_indoor_temp, data.orig_outdoor_temp
        """Choose an action based on minimizing predicted free energy."""
        best_action = 0
        lowest_free_energy = float('inf')

        for potential_action in np.linspace(-1, 1, 21):  # considering 21 potential actions from -1 to 1
            predicted = model(
                torch.FloatTensor([[orig_indoor_temp, orig_outdoor_temp, potential_action]]).unsqueeze(0)
            )
            predicted_temp = predicted[0,0,0]
            fe = self.free_energy(
                predicted = predicted_temp,
                observed  = torch.tensor(self.optimal_temp, dtype=torch.float32)
            )
            if fe.item() < lowest_free_energy:
                best_action = potential_action
                lowest_free_energy = fe.item()

        return best_action, predicted[0, 0, 0], fe.detach()

# Initialize Environment
env       = Environment(outdoor_temps, init_temp=20.0)
model     = ThermostatRNN(env.TEMPERATURE_GOAL, lr=0.001)
states = []

 
for step in range(10000):
    data = TimeStepData(step, env.curr_indoor_temp, env.curr_outdoor_temp)
    
    # Choose action 
    data.action, data.pred_indoor_temp, data.free_energy = \
        model.choose_action(data)
    data.power = data.action * env.ACTION_SCALE
    
    # Update state, and check accuracy
    env.run_timestep(data.power)
    data.next_indoor_temp = torch.tensor(env.curr_indoor_temp, dtype=torch.float32)
    model.train_step(data)
    
    # Print 
    if step % 1000 == 0:
        print(f"Step {step+1}:",
              f"Current Temp: {data.orig_indoor_temp:.2f},",
              f"Outdoor Temp: {data.orig_outdoor_temp:.2f},",
              f"Chosen Action: {data.action:.2f}, Power: {data.power:.2f}"
        )
    states.append(data.detach())
    
states = np.array(states)

import matplotlib.pyplot as plt

# Assuming states is a numpy array or similar with the described structure
# Initialize a figure with 2 subplots
fig, ax = plt.subplots(3, 1, figsize=(10, 8))

# plot 1: temperatures on the first subplot (ax[0])
ax[0].plot([d.orig_outdoor_temp for d in states], label="outdoor_temp")
ax[0].plot([d.next_indoor_temp  for d in states], label="current_temp")
ax[0].plot([d.pred_indoor_temp  for d in states], label="pred")
ax[0].plot([0, len(states)], [env.TEMPERATURE_GOAL, env.TEMPERATURE_GOAL], ":")
ax[0].set_title("Temperatures")
ax[0].legend()

# plot 2: action on the second subplot (ax[1])
ax[1].plot([d.action for d in states], label="action")
ax[1].set_title("Action")
ax[1].legend()

# plot 3: Free Energy
ax[2].semilogy([d.free_energy for d in states], label="free energy")
ax[2].set_title("Free Energy")
ax[2].legend()

plt.tight_layout()
plt.show()